In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_io as tfio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tf_clahe
import os
import cv2
print(tf.__version__)

2.6.2


In [2]:
def bcet(img):
    Lmin = tf.reduce_min(img) # MINIMUM OF INPUT IMAGE
#     Lmin = np.min(img) # MINIMUM OF INPUT IMAGE
#     print("Lmin", Lmin)
    Lmax = tf.reduce_max(img) # MAXIMUM OF INPUT IMAGE
#     Lmax = np.max(img) # MAXIMUM OF INPUT IMAGE
#     print("Lmax", Lmax)
    Lmean = tf.reduce_mean(img) #MEAN OF INPUT IMAGE
#     Lmean = np.mean(img) #MEAN OF INPUT IMAGE
#     print("Lmean", Lmean)
    LMssum = tf.reduce_mean(img * img) #MEAN SQUARE SUM OF INPUT IMAGE
#     LMssum = np.mean(pow(img, 2)) #MEAN SQUARE SUM OF INPUT IMAGE
#     print("LMssum", LMssum)

    Gmin = tf.constant(0, dtype="float32") #MINIMUM OF OUTPUT IMAGE
    Gmax = tf.constant(255, dtype="float32") #MAXIMUM OF OUTPUT IMAGE
    Gmean = tf.constant(110, dtype="float32") #MEAN OF OUTPUT IMAGE
    
    subber = tf.constant(2, dtype="float32")
    
    # find b
    
    bnum = ((Lmax**subber)*(Gmean-Gmin)) - (LMssum*(Gmax-Gmin)) + ((Lmin**subber) *(Gmax-Gmean))
    bden = subber * ((Lmax*(Gmean-Gmin)) - (Lmean*(Gmax-Gmin)) + (Lmin*(Gmax-Gmean)))
    
    b = bnum/bden
    
    # find a
    a1 = Gmax-Gmin
    a2 = Lmax-Lmin
    a3 = Lmax+Lmin-(subber*b)
            
    a = a1/(a2*a3)
    
    # find c
    c = Gmin - (a*(Lmin-b)**subber)
    
    # Process raster
    y = a*((img - b)**subber) + c #PARABOLIC FUNCTION
    
    final = tf.clip_by_value(y, clip_value_min=0, clip_value_max=255)
    return y

In [3]:
def bcet_processing(img,channels=3):
#     img = tf.make_tensor_proto(img,dtype="int64")
#     img = tf.make_ndarray(img)
#     print(img.shape)
    layers = []
    for i in range(channels):
        layer = img[:,:,i]
        layer = bcet(layer)
        layers.append(layer)
        
# #     print(red.shape)
#     blue = img[:,:,1]
# #     print(blue.shape)
#     green = img[:,:,2]
#     red = img[:,:,0]
# #     print(red.shape)
#     blue = img[:,:,1]
# #     print(blue.shape)
#     green = img[:,:,2]
# #     print(green.shape)
    
#     red = bcet(red)
# #     print(red.shape)
#     blue = bcet(blue)
# #     print(blue.shape)
#     green = bcet(green)
# #     print(green.shape)
    
#     final_image = np.stack((red, blue, green), axis=-1)
#     final_image = tf.convert_to_tensor(final_image, dtype=tf.int64) 
#     print(final_image.shape)
    final_image = tf.stack(layers, axis=-1)
#     print(final_image.shape)
    return final_image

In [4]:


def convert_file_bcet(inputFile, outputFile):

    img = tf.io.read_file(inputFile)
    img = tf.io.decode_bmp(img, channels=3)
    # print(tf.rank(img))
    img = tf.cast(img, tf.float32)

    
    img = bcet_processing(img)
    
    tf.keras.utils.save_img(outputFile, img)
#     plt.figure()
#     img = tf.cast(img, tf.int32)
#     plt.imshow(img)
# #     # plt.savefig('normal_bcet.png')
#     plt.savefig(outputFile, format="jpg")   # save the figure to file
#     plt.close() 

def convert_file_clahe(inputFile, outputFile):
    bgr = cv2.imread(inputFile)

    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)

    lab_planes = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(5,5))

    eq_channels = []
    for ch in lab_planes:
        eq_channels.append(clahe.apply(ch))

    eq_image = cv2.merge(eq_channels)

    # lab = cv2.merge(lab_planes)

    bgr = cv2.cvtColor(eq_image, cv2.COLOR_LAB2BGR)
    cv2.imwrite(outputFile, bgr)


def convert_custom_method(inputFile, outputFile):
    img = tf.io.read_file(inputFile)
    img = tf.io.decode_bmp(img, channels=3)
    # print(tf.rank(img))
    img = tf.cast(img, tf.float32)

    img = tfio.experimental.color.rgb_to_bgr(img)
    img = tf.image.adjust_contrast(img, 11.)
    img = tf.image.adjust_hue(img, 11.)
    img = tf.image.adjust_gamma(img)
    img = tfa.image.median_filter2d(img)
    
    tf.keras.utils.save_img(outputFile, img)

def convert_image_to_bmp(inputFile, ext):
    
    if inputFile.endswith(ext): 
        prefix = inputFile.split(ext)[0]
        im = Image.open(inputFile)
        im.save(prefix + '.bmp')  

In [11]:
## convert colour of images

Input_dir = 'mura_data/RGB/train_data_nctu_v2/defect/'
Out_dir = 'mura_data/RGB/clahe_mid_train_data_nctu_v2/defect/'
a = os.listdir(Input_dir)
for i in a:
    if i != ".DS_Store":
        
        inputFile = Input_dir+i
        OutputFile = Out_dir+i
        
        # print(inputFile)
        # convert_file_bcet(inputFile, OutputFile)
        convert_file_clahe(inputFile, OutputFile)
        # convert_file_clahe_v2(inputFile, OutputFile)
        # convert_image_to_bmp(inputFile, ".png")
        # convert_custom_method(inputFile, OutputFile)
print("done")
#     break

done
